In [1]:
from ultralytics import YOLO
import torch
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
import shutil
from glob import glob

In [2]:
base_path = '../../dataset/sagittal'
train_image = os.path.join(base_path, 'images/sagittal_train_CLAHE')
train_label = os.path.join(base_path, 'labels/train')
test_image = os.path.join(base_path, 'images/test')
test_label = os.path.join(base_path, 'labels/test')
val_image = os.path.join(base_path, 'images/val')
val_label = os.path.join(base_path, 'labels/val')

In [3]:
# seed 및 GPU 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(42)

# Read and sort the file list to ensure consistent order
file_list = sorted(os.listdir(train_image))
random_file = random.choice(file_list).split('.')[0]
print("Image :", random_file)

# Read labels from the file
with open(os.path.join(train_label, f'{random_file}.txt'), 'r') as f:
    labels = f.readlines()
    labels = labels[0].split(' ')

# Read and process the image
img = cv2.imread(os.path.join(train_image, f'{random_file}.jpg'), cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Image : 00166_259


In [4]:
# 파일 개수 세는 함수
def count_files(directory):
    return len(os.listdir(directory))

num_train_images = count_files(train_image)
num_test_images = count_files(test_image)
num_train_labels = count_files(train_label)
num_test_labels = count_files(test_label)
print(f'Train images: {num_train_images}, Train labels: {num_train_labels}')
print(f'Test images: {num_test_images}, Test labels: {num_test_labels}\n')

Train images: 212, Train labels: 212
Test images: 70, Test labels: 70



In [11]:
#v8n
experiment_name = 'train'
model_size = 'v8n' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_n = YOLO('yolov8n.pt') #이름수정주의 2개
results_n = model_n.train( #이름수정주의 2개
    data='CLAHE.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

New https://pypi.org/project/ultralytics/8.2.41 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=CLAHE.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8n, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

train: Scanning /home/under1/Detect/dataset/sagittal/labels/sagittal_train_CLAHE... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<00:00, 895.25it/s]

train: New cache created: /home/under1/Detect/dataset/sagittal/labels/sagittal_train_CLAHE.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/home/under1/.conda/envs/detect/lib/python3.8/site-packages/albumentations/core/composition.py:144: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to runs/train/train_v8n/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/train/train_v8n
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    200/200       2.2G     0.3898     0.2718     0.8182          4        640: 100%|██████████| 14/14 [00:01<00:00, 12.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 11.90it/s]

                   all         52         54      0.508      0.729      0.565      0.438



200 epochs completed in 0.157 hours.
Optimizer stripped from runs/train/train_v8n/weights/last.pt, 6.3MB
Optimizer stripped from runs/train/train_v8n/weights/best.pt, 6.3MB

Validating runs/train/train_v8n/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.71it/s]


                   all         52         54      0.489      0.723      0.643       0.49
              negative         52         30      0.556      0.867      0.688      0.546
              positive         52         24      0.422      0.579      0.598      0.433
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/train/train_v8n


In [12]:
result_n = model_n.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.15it/s]


                   all         52         54      0.489      0.723      0.643      0.487
              negative         52         30      0.555      0.867      0.687       0.54
              positive         52         24      0.423       0.58        0.6      0.434
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/train_v8n
